In [1]:
import os
import pandas as pd
import numpy as np
import time, datetime
import tsfel

In [2]:
def make_windows(data, windowsize, overlap):
    ln = len(data)
    window = windowsize
    stepsize = (1 - overlap) * window
    start = 0
    end = window
    
    slices = []
    while end < len(data):
        slices.append((start, end))
        start += stepsize
        end += stepsize    
        
    return np.array(slices, dtype=np.int32)

In [3]:
def split_day(t2):
    t3 = time.localtime(t2)
    t4 = time.strftime('%Y-%m-%d %H:%M:%S', t3)
    t5 = t4[0:10] + ' 00:00:00'
    t6 = time.strptime(t5, "%Y-%m-%d %H:%M:%S")
    t7 = int(time.mktime(t6))
    t_start = t7
    t_end = t7 + 60 * 60 * 24
    
    return t_start, t_end

In [4]:
def gen_feature_extraction(signal, fs):
    extracted_features = []
    
    max_v = tsfel.feature_extraction.features.calc_max(signal)
    min_v = tsfel.feature_extraction.features.calc_min(signal)
    mean_v = tsfel.feature_extraction.features.calc_mean(signal)
    median_v = tsfel.feature_extraction.features.calc_median(signal)
    std_v = tsfel.feature_extraction.features.calc_std(signal)
    var_v = tsfel.feature_extraction.features.calc_var(signal)
      
    extracted_features.append(max_v)
    extracted_features.append(min_v)
    extracted_features.append(mean_v)
    extracted_features.append(median_v)
    extracted_features.append(std_v)
    extracted_features.append(var_v)
    
    return extracted_features

In [5]:
def gen_window_feature_extraction(data, windowsize, overlap, window_statues, total_features):
    sample_rate = 1
    
    if window_statues:
        windows = make_windows(data, windowsize, overlap)
        windows_index = 1
        for window in windows:
            window_features = []
            window_features.append('week ' + str(windows_index))
            windows_index += 1
            start = window[0]
            end = window[1]
            window_data = data[start: end]
            signal = np.array(window_data['value'].tolist())
            window_features.append(window_data['dateTime'].iloc[0])
            window_features.append(window_data['device_id'].iloc[0])
            window_features.extend(gen_feature_extraction(signal, sample_rate))
            total_features.append(window_features) 
        
    else:
        signal = np.array(data)
        window_features = gen_feature_extraction(signal, sample_rate)
        total_features.append(window_features) 
 
    return total_features

In [6]:
def gen_main(df, windowsize, overlap, window_statues):
    total_features = []
    features_cols = ['week', 
                     'time', 
                     'device_id', 
                     'max', 
                     'min', 
                     'mean', 
                     'median', 
                     'std', 
                     'var']
    df_groups = df.groupby('fitbit_id')
    df_groups_names = df_groups.size().index
    for df_groups_name in df_groups_names:
        df_group = df_groups.get_group(df_groups_name)
        df_group_new = []
        i = 0
        while i < len(df_group):
            t1 = df_group['time'].iloc[i]
            t2 = int(str(t1)[0:10])
            start_t2, end_t2 = split_day(t2)
            j = i + 1
            df_day = df_group['value'].iloc[i]
            device_id = df_group['fitbit_id'].iloc[i]
            while j < len(df_group):
                t1_next = df_group['time'].iloc[j]
                t2_next = int(str(t1_next)[0:10])
                if t2_next < end_t2:
                    df_day += df_group['value'].iloc[j]
                    j += 1
                else:
                    t3 = time.localtime(start_t2)
                    t4 = time.strftime('%Y-%m-%d', t3)
                    df_group_new.append([t4, df_day, device_id])
                    break
                
                while(t2_next >= end_t2 + 60 * 60 * 24):
                    t3 = time.localtime(end_t2)
                    t4 = time.strftime('%Y-%m-%d', t3)
                    df_group_new.append([t4, np.nan, device_id])
                    end_t2 += 60 * 60 * 24
            
            i = j
        
        t3 = time.localtime(start_t2)
        t4 = time.strftime('%Y-%m-%d', t3)
        df_group_new.append([t4, df_day])
        df_group_new = pd.DataFrame(df_group_new, columns = ['dateTime', 'value', 'device_id'])
        total_features = gen_window_feature_extraction(df_group_new, windowsize, overlap, window_statues, total_features)
    
    df_features = pd.DataFrame(total_features, columns = features_cols)
    
    return df_features

In [7]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
step = pd.read_csv('activities-steps.csv')
step.head()

,name,time,dateTime,fitbit_id,value
0,activities-steps,1579219200000000000,2020-01-17,7WL5F5,2438
1,activities-steps,1579237200000000000,2020-01-17,58WZR9,4998
2,activities-steps,1579237200000000000,2020-01-17,63YVWZ,0
3,activities-steps,1579237200000000000,2020-01-17,7VZ2W8,0
4,activities-steps,1579237200000000000,2020-01-17,7W8B9J,0


In [8]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
step_features = gen_main(step, 7, 0, True)
step_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,11616,3342,7158.285714,6632.0,2713.420862,7.362653e+06
1,week 2,2020-01-24,58WZR9,13225,4480,8840.714286,8901.0,2834.220866,8.032808e+06
2,week 3,2020-01-31,58WZR9,11514,7166,8580.000000,8002.0,1419.879673,2.016058e+06
3,week 4,2020-02-07,58WZR9,14220,8223,12159.285714,12597.0,1837.958939,3.378093e+06
4,week 5,2020-02-14,58WZR9,13662,5603,9791.142857,8354.0,2834.105323,8.032153e+06


In [9]:
step_features.to_csv('activities-steps-features.csv')

In [51]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
step_intra = pd.read_csv('activities-steps-intraday.csv')
step_intra.head()

,name,time,fitbit_id,value
0,activities-steps-intraday,1581465600000000000,58WZR9,0
1,activities-steps-intraday,1581465600000000000,63YVWZ,0
2,activities-steps-intraday,1581465600000000000,7VZ2W8,0
3,activities-steps-intraday,1581465600000000000,7W8BY3,0
4,activities-steps-intraday,1581465600000000000,7W8LJH,0


In [52]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
step_intra_features = gen_main(step_intra, 7, 0, True)
step_intra_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-02-11,58WZR9,14174,12,6277.142857,6145.0,4938.917534,2.439291e+07
1,week 2,2020-02-23,58WZR9,8245,0,2314.142857,461.0,2880.083254,8.294880e+06
2,week 3,2020-03-01,58WZR9,9821,0,2677.142857,0.0,4236.190621,1.794531e+07
3,week 4,2020-05-06,58WZR9,10987,5424,8571.428571,8197.0,1698.100271,2.883545e+06
4,week 5,2020-05-13,58WZR9,10045,1716,5639.285714,5525.0,3204.975583,1.027187e+07


In [53]:
step_intra_features.to_csv('activities-steps-intraday-features.csv')

In [11]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
calor = pd.read_csv('activities-calories.csv')
calor.head()

,name,time,dateTime,fitbit_id,value
0,activities-calories,1579219200000000000,2020-01-17,7WL5F5,2045
1,activities-calories,1579237200000000000,2020-01-17,58WZR9,1501
2,activities-calories,1579237200000000000,2020-01-17,63YVWZ,1412
3,activities-calories,1579237200000000000,2020-01-17,7VZ2W8,1670
4,activities-calories,1579237200000000000,2020-01-17,7W8B9J,1679


In [12]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
calor_features = gen_main(calor, 7, 0, True)
calor_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,2020,1432,1637.714286,1591.0,179.966437,32387.918367
1,week 2,2020-01-24,58WZR9,2047,1458,1671.142857,1663.0,176.236390,31059.265306
2,week 3,2020-01-31,58WZR9,1769,1560,1665.857143,1700.0,85.157717,7251.836735
3,week 4,2020-02-07,58WZR9,1896,1691,1793.571429,1766.0,72.555116,5264.244898
4,week 5,2020-02-14,58WZR9,1974,1561,1722.142857,1626.0,152.702726,23318.122449


In [13]:
calor_features.to_csv('activities-calories-features.csv')

In [14]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
BMR = pd.read_csv('activities-caloriesBMR.csv')
BMR.head()

,name,time,dateTime,fitbit_id,value
0,activities-caloriesBMR,1579219200000000000,2020-01-17,7WL5F5,1666
1,activities-caloriesBMR,1579237200000000000,2020-01-17,58WZR9,1155
2,activities-caloriesBMR,1579237200000000000,2020-01-17,63YVWZ,1412
3,activities-caloriesBMR,1579237200000000000,2020-01-17,7VZ2W8,1670
4,activities-caloriesBMR,1579237200000000000,2020-01-17,7W8B9J,1679


In [15]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
BMR_features = gen_main(BMR, 7, 0, True)
BMR_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,1155,1155,1155.0,1155.0,0.0,0.0
1,week 2,2020-01-24,58WZR9,1155,1155,1155.0,1155.0,0.0,0.0
2,week 3,2020-01-31,58WZR9,1155,1155,1155.0,1155.0,0.0,0.0
3,week 4,2020-02-07,58WZR9,1155,1155,1155.0,1155.0,0.0,0.0
4,week 5,2020-02-14,58WZR9,1155,1155,1155.0,1155.0,0.0,0.0


In [16]:
BMR_features.to_csv('activities-caloriesBMR-features.csv')

In [17]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
calor_intra = pd.read_csv('activities-calories-intraday.csv')
calor_intra.head()

,name,time,fitbit_id,level,mets,value
0,activities-calories-intraday,1581465600000000000,58WZR9,0,10,0.8027
1,activities-calories-intraday,1581465600000000000,63YVWZ,0,10,0.9811
2,activities-calories-intraday,1581465600000000000,7VZ2W8,0,10,1.1601
3,activities-calories-intraday,1581465600000000000,7W8BY3,0,10,1.0179
4,activities-calories-intraday,1581465600000000000,7W8LJH,0,10,1.0345


In [18]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
calor_intra_features = gen_main(calor_intra, 7, 0, True)
calor_intra_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-02-11,58WZR9,1714.807989,243.539175,1146.381722,1488.125507,586.406524,343872.611616
1,week 2,2020-02-23,58WZR9,1553.144207,1110.936777,1281.476126,1153.078527,186.880687,34924.391209
2,week 3,2020-03-01,58WZR9,1634.136637,195.216636,1060.687761,1110.936777,419.504386,175983.930133
3,week 4,2020-05-06,58WZR9,1714.486912,1495.189268,1589.942269,1583.566538,69.852467,4879.367186
4,week 5,2020-05-13,58WZR9,1676.278385,1286.647805,1450.043124,1420.698704,147.165370,21657.645999


In [19]:
calor_intra_features.to_csv('activities-calories-intraday-features.csv')

In [20]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
dis = pd.read_csv('activities-distance.csv')
dis.head()

,name,time,dateTime,fitbit_id,value
0,activities-distance,1579219200000000000,2020-01-17,7WL5F5,1.82299
1,activities-distance,1579237200000000000,2020-01-17,58WZR9,3.33635
2,activities-distance,1579237200000000000,2020-01-17,63YVWZ,0.00000
3,activities-distance,1579237200000000000,2020-01-17,7VZ2W8,0.00000
4,activities-distance,1579237200000000000,2020-01-17,7W8B9J,0.00000


In [21]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
dis_features = gen_main(dis, 7, 0, True)
dis_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,9.34725,2.05621,5.342417,4.39693,2.552330,6.514391
1,week 2,2020-01-24,58WZR9,10.78430,3.20296,6.678443,6.41146,2.520635,6.353600
2,week 3,2020-01-31,58WZR9,8.70182,5.19626,6.385304,6.55887,1.175530,1.381872
3,week 4,2020-02-07,58WZR9,10.50327,5.50105,8.647583,8.69660,1.503386,2.260169
4,week 5,2020-02-14,58WZR9,10.09167,3.71938,7.250637,6.80485,2.367886,5.606885


In [22]:
dis_features.to_csv('activities-distance-features.csv')

In [23]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
dis_intra = pd.read_csv('activities-distance-intraday.csv')
dis_intra.head()

,name,time,fitbit_id,value
0,activities-distance-intraday,1581465600000000000,58WZR9,0.0
1,activities-distance-intraday,1581465600000000000,63YVWZ,0.0
2,activities-distance-intraday,1581465600000000000,7VZ2W8,0.0
3,activities-distance-intraday,1581465600000000000,7W8BY3,0.0
4,activities-distance-intraday,1581465600000000000,7W8LJH,0.0


In [24]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
dis_intra_features = gen_main(dis_intra, 7, 0, True)
dis_intra_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-02-11,58WZR9,10.2095,0.0077,4.647471,4.2626,3.812076,14.531921
1,week 2,2020-02-23,58WZR9,6.9005,0.0000,1.721129,0.2980,2.343102,5.490127
2,week 3,2020-03-01,58WZR9,8.0340,0.0000,1.970186,0.0000,3.172014,10.061670
3,week 4,2020-05-06,58WZR9,7.7044,3.4930,6.281114,6.4909,1.220913,1.490628
4,week 5,2020-05-13,58WZR9,7.5514,1.1083,4.015386,4.4510,2.395392,5.737902


In [25]:
dis_intra_features.to_csv('activities-distance-intraday-features.csv')

In [26]:
data_path ="C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
ele_intra = pd.read_csv('activities-elevation-intraday.csv')
ele_intra.head()

,name,time,fitbit_id,value
0,activities-elevation-intraday,1581465600000000000,7WL5F5,0
1,activities-elevation-intraday,1581465660000000000,7WL5F5,0
2,activities-elevation-intraday,1581465720000000000,7WL5F5,0
3,activities-elevation-intraday,1581465780000000000,7WL5F5,0
4,activities-elevation-intraday,1581465840000000000,7WL5F5,0


In [31]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
ele_intra_features = gen_main(ele_intra, 7, 0, True)
ele_intra_features.head()

,week,time,device_id,max,min,mean,median,std,var


In [32]:
ele_intra_features.to_csv('activities-elevation-intraday-features.csv')

In [33]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
floor_intra = pd.read_csv('activities-floors-intraday.csv')
floor_intra.head()

,name,time,fitbit_id,value
0,activities-floors-intraday,1581465600000000000,7WL5F5,0
1,activities-floors-intraday,1581465660000000000,7WL5F5,0
2,activities-floors-intraday,1581465720000000000,7WL5F5,0
3,activities-floors-intraday,1581465780000000000,7WL5F5,0
4,activities-floors-intraday,1581465840000000000,7WL5F5,0


In [34]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
floor_intra_features = gen_main(floor_intra, 7, 0, True)
floor_intra_features.head()

,week,time,device_id,max,min,mean,median,std,var


In [35]:
floor_intra_features.to_csv('activities-floor-intraday-features.csv')

##   To many missing data for hr

In [37]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
hr = pd.read_csv('activities-heart.csv')
hr.head()

,name,time,Cardio_caloriesOut,Cardio_max,Cardio_min,Cardio_minutes,Fat_Burn_caloriesOut,Fat_Burn_max,Fat_Burn_min,Fat_Burn_minutes,Out_of_Range_caloriesOut,Out_of_Range_max,Out_of_Range_min,Out_of_Range_minutes,Peak_caloriesOut,Peak_max,Peak_min,Peak_minutes,fitbit_id,restingHeartRate
0,activities-heart,1579237200000000000,18.86345,162,133,3.0,186.78829,133,95,67.0,1271.23599,95,30,1347.0,0.0,220,162,0.0,58WZR9,63.0
1,activities-heart,1579237200000000000,NaN,161,133,NaN,NaN,133,95,NaN,NaN,95,30,NaN,NaN,220,161,NaN,7WL8F9,NaN
2,activities-heart,1579237200000000000,NaN,161,133,NaN,NaN,133,95,NaN,NaN,95,30,NaN,NaN,220,161,NaN,7W8BY3,NaN
3,activities-heart,1579237200000000000,NaN,161,133,NaN,NaN,133,95,NaN,NaN,95,30,NaN,NaN,220,161,NaN,7W8MMN,NaN
4,activities-heart,1579237200000000000,28.69360,146,120,3.0,136.52600,120,86,31.0,1110.48860,86,30,742.0,0.0,220,146,0.0,7WL5F5,54.0


In [38]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
factive = pd.read_csv('activities-minutesFairlyActive.csv')
factive.head()

,name,time,dateTime,fitbit_id,value
0,activities-minutesFairlyActive,1579219200000000000,2020-01-17,7WL5F5,0
1,activities-minutesFairlyActive,1579237200000000000,2020-01-17,58WZR9,5
2,activities-minutesFairlyActive,1579237200000000000,2020-01-17,63YVWZ,0
3,activities-minutesFairlyActive,1579237200000000000,2020-01-17,7VZ2W8,0
4,activities-minutesFairlyActive,1579237200000000000,2020-01-17,7W8B9J,0


In [39]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
factive_features = gen_main(factive, 7, 0, True)
factive_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,35,0,8.142857,5.0,11.166641,124.693878
1,week 2,2020-01-24,58WZR9,48,0,13.857143,9.0,15.366276,236.122449
2,week 3,2020-01-31,58WZR9,26,0,6.285714,4.0,8.241260,67.918367
3,week 4,2020-02-07,58WZR9,29,8,16.000000,14.0,6.824326,46.571429
4,week 5,2020-02-14,58WZR9,23,0,7.000000,3.0,7.578164,57.428571


In [40]:
factive_features.to_csv('activities-minutesFairlyActive-features.csv')

In [41]:
data_path ="C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
lactive = pd.read_csv('activities-minutesLightlyActive.csv')
lactive.head()

,name,time,dateTime,fitbit_id,value
0,activities-minutesLightlyActive,1579219200000000000,2020-01-17,7WL5F5,127
1,activities-minutesLightlyActive,1579237200000000000,2020-01-17,58WZR9,143
2,activities-minutesLightlyActive,1579237200000000000,2020-01-17,63YVWZ,0
3,activities-minutesLightlyActive,1579237200000000000,2020-01-17,7VZ2W8,0
4,activities-minutesLightlyActive,1579237200000000000,2020-01-17,7W8B9J,0


In [42]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
lactive_features = gen_main(lactive, 7, 0, True)
lactive_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,236,143,179.428571,156.0,36.858250,1358.530612
1,week 2,2020-01-24,58WZR9,226,130,171.142857,159.0,36.317759,1318.979592
2,week 3,2020-01-31,58WZR9,296,135,190.000000,167.0,52.098807,2714.285714
3,week 4,2020-02-07,58WZR9,245,164,205.000000,209.0,25.623650,656.571429
4,week 5,2020-02-14,58WZR9,247,157,202.142857,199.0,33.917667,1150.408163


In [43]:
lactive_features.to_csv('activities-minutesLightlyActive-features.csv')

In [44]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
sactive = pd.read_csv('activities-minutesSedentary.csv')
sactive.head()

,name,time,dateTime,fitbit_id,value
0,activities-minutesSedentary,1579219200000000000,2020-01-17,7WL5F5,1313
1,activities-minutesSedentary,1579237200000000000,2020-01-17,58WZR9,782
2,activities-minutesSedentary,1579237200000000000,2020-01-17,63YVWZ,1440
3,activities-minutesSedentary,1579237200000000000,2020-01-17,7VZ2W8,1440
4,activities-minutesSedentary,1579237200000000000,2020-01-17,7W8B9J,1440


In [45]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
sactive_features = gen_main(sactive, 7, 0, True)
sactive_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,869,452,690.428571,733.0,138.756577,19253.387755
1,week 2,2020-01-24,58WZR9,934,600,737.428571,733.0,99.985509,9997.102041
2,week 3,2020-01-31,58WZR9,823,603,706.000000,702.0,74.123449,5494.285714
3,week 4,2020-02-07,58WZR9,825,734,751.714286,739.0,30.150642,909.061224
4,week 5,2020-02-14,58WZR9,848,647,726.142857,740.0,73.181630,5355.551020


In [46]:
sactive_features.to_csv('activities-minutesSedentary-features.csv')

In [47]:
data_path = "C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\fitbit_0810\\"
os.chdir(data_path)
mactive = pd.read_csv('activities-minutesVeryActive.csv')
mactive.head()

,name,time,dateTime,fitbit_id,value
0,activities-minutesVeryActive,1579219200000000000,2020-01-17,7WL5F5,0
1,activities-minutesVeryActive,1579237200000000000,2020-01-17,58WZR9,3
2,activities-minutesVeryActive,1579237200000000000,2020-01-17,63YVWZ,0
3,activities-minutesVeryActive,1579237200000000000,2020-01-17,7VZ2W8,0
4,activities-minutesVeryActive,1579237200000000000,2020-01-17,7W8B9J,0


In [48]:
feature_path = 'C:\\Users\\huzec\\Desktop\\Doryab\\fitbit\\features\\'
os.chdir(feature_path)
mactive_features = gen_main(mactive, 7, 0, True)
mactive_features.head()

,week,time,device_id,max,min,mean,median,std,var
0,week 1,2020-01-17,58WZR9,48,0,18.428571,10.0,18.274550,333.959184
1,week 2,2020-01-24,58WZR9,55,6,26.714286,20.0,16.883756,285.061224
2,week 3,2020-01-31,58WZR9,33,0,20.857143,19.0,10.642617,113.265306
3,week 4,2020-02-07,58WZR9,46,15,29.857143,29.0,9.295599,86.408163
4,week 5,2020-02-14,58WZR9,61,0,30.571429,30.0,17.654276,311.673469


In [49]:
mactive_features.to_csv('activities-minutesVeryActive-features.csv')